<a href="https://colab.research.google.com/github/youkyung2/dataAnalysis/blob/master/%ED%9A%8C%EA%B7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# 회귀
# 보스턴집값문제
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.datasets import load_boston

boston = load_boston()
bostonDF = pd.DataFrame(boston.data, columns = boston.feature_names)
bostonDF.head(3)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03


In [4]:
bostonDF['PRICE'] = boston.target
bostonDF.head(3)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7


In [7]:
# EDA
bostonDF.info()
bostonDF.describe()
bostonDF.corr()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float64
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  PRICE    506 non-null    float64
dtypes: float64(14)
memory usage: 55.5 KB


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
CRIM,1.000000,-0.200469,0.406583,-0.055892,0.420972,-0.219247,0.352734,-0.379670,0.625505,0.582764,0.289946,-0.385064,0.455621,-0.388305
ZN,-0.200469,1.000000,-0.533828,-0.042697,-0.516604,0.311991,-0.569537,0.664408,-0.311948,-0.314563,-0.391679,0.175520,-0.412995,0.360445
INDUS,0.406583,-0.533828,1.000000,0.062938,0.763651,-0.391676,0.644779,-0.708027,0.595129,0.720760,0.383248,-0.356977,0.603800,-0.483725
CHAS,-0.055892,-0.042697,0.062938,1.000000,0.091203,0.091251,0.086518,-0.099176,-0.007368,-0.035587,-0.121515,0.048788,-0.053929,0.175260
NOX,0.420972,-0.516604,0.763651,0.091203,1.000000,-0.302188,0.731470,-0.769230,0.611441,0.668023,0.188933,-0.380051,0.590879,-0.427321
RM,-0.219247,0.311991,-0.391676,0.091251,-0.302188,1.000000,-0.240265,0.205246,-0.209847,-0.292048,-0.355501,0.128069,-0.613808,0.695360
AGE,0.352734,-0.569537,0.644779,0.086518,0.731470,-0.240265,1.000000,-0.747881,0.456022,0.506456,0.261515,-0.273534,0.602339,-0.376955
DIS,-0.379670,0.664408,-0.708027,-0.099176,-0.769230,0.205246,-0.747881,1.000000,-0.494588,-0.534432,-0.232471,0.291512,-0.496996,0.249929
RAD,0.625505,-0.311948,0.595129,-0.007368,0.611441,-0.209847,0.456022,-0.494588,1.000000,0.910228,0.464741,-0.444413,0.488676,-0.381626
TAX,0.582764,-0.314563,0.720760,-0.035587,0.668023,-0.292048,0.506456,-0.534432,0.910228,1.000000,0.460853,-0.441808,0.543993,-0.468536


In [16]:
# RM 과 LSTAT 이 강한상관관계를 보이므로, 해당 칼럼의 이상치 조사
def get_outlier(df, col) : 
  IQR = df[col].quantile(0.75) - df[col].quantile(0.25)
  minimum = df[col].quantile(0.25) - (1.5 * IQR)
  maximum = df[col].quantile(0.75) + (1.5 * IQR)

  return df[(df[col] < minimum) | (df[col] > maximum)].index

In [21]:
print(get_outlier(bostonDF, 'RM'))
print(get_outlier(bostonDF, 'LSTAT'))
print(bostonDF.iloc[get_outlier(bostonDF, 'RM') & get_outlier(bostonDF, 'LSTAT')])

Int64Index([ 97,  98, 162, 163, 166, 180, 186, 195, 203, 204, 224, 225, 226,
            232, 233, 253, 257, 262, 267, 280, 283, 364, 365, 367, 374, 384,
            386, 406, 412, 414],
           dtype='int64')
Int64Index([141, 373, 374, 387, 412, 414, 438], dtype='int64')
        CRIM   ZN  INDUS  CHAS    NOX  ...    TAX  PTRATIO       B  LSTAT  PRICE
374  18.4982  0.0   18.1   0.0  0.668  ...  666.0     20.2  396.90  37.97   13.8
412  18.8110  0.0   18.1   0.0  0.597  ...  666.0     20.2   28.79  34.37   17.9
414  45.7461  0.0   18.1   0.0  0.693  ...  666.0     20.2   88.27  36.98    7.0

[3 rows x 14 columns]


In [24]:
bostonDF.tail(3)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0
505,0.04741,0.0,11.93,0.0,0.573,6.030,80.8,2.5050,1.0,273.0,21.0,396.90,7.88,11.9


In [25]:
bostonDF.drop(index = [374,412,414], inplace=True)

In [28]:
dir(sklearn)

['__SKLEARN_SETUP__',
 '__all__',
 '__builtins__',
 '__cached__',
 '__check_build',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_config',
 '_distributor_init',
 'base',
 'clone',
 'config_context',
 'datasets',
 'exceptions',
 'externals',
 'feature_extraction',
 'get_config',
 'logger',
 'logging',
 'metrics',
 'model_selection',
 'os',
 'preprocessing',
 're',
 'set_config',
 'setup_module',
 'show_versions',
 'sys',
 'utils']

In [33]:
help(sklearn)

Help on package sklearn:

NAME
    sklearn

DESCRIPTION
    Machine learning module for Python
    
    sklearn is a Python module integrating classical machine
    learning algorithms in the tightly-knit world of scientific Python
    packages (numpy, scipy, matplotlib).
    
    It aims to provide simple and efficient solutions to learning problems
    that are accessible to everybody and reusable in various contexts:
    machine-learning as a versatile tool for science and engineering.
    
    See http://scikit-learn.org for complete documentation.

PACKAGE CONTENTS
    __check_build (package)
    _build_utils (package)
    _config
    _distributor_init
    _isotonic
    base
    calibration
    cluster (package)
    compose (package)
    conftest
    covariance (package)
    cross_decomposition (package)
    datasets (package)
    decomposition (package)
    discriminant_analysis
    dummy
    ensemble (package)
    exceptions
    experimental (package)
    externals (package)
   

In [42]:
x = bostonDF.drop('PRICE', axis=1, inplace=False)
x.head(3)

y = bostonDF['PRICE']
y.head(3)

0    24.0
1    21.6
2    34.7
Name: PRICE, dtype: float64

In [43]:
# 정규화
import sklearn.preprocessing
dir(sklearn.preprocessing)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
scaled_x = sc.fit_transform(x)
scaled_x

array([[-0.41448315,  0.28128361, -1.28199672, ..., -1.45269358,
         0.43720271, -1.08943139],
       [-0.41197133, -0.48952202, -0.58738866, ..., -0.29797665,
         0.43720271, -0.4876629 ],
       [-0.41197373, -0.48952202, -0.58738866, ..., -0.29797665,
         0.3917172 , -1.22685449],
       ...,
       [-0.40796847, -0.48952202,  0.121812  , ...,  1.18006101,
         0.43720271, -0.99395851],
       [-0.40212513, -0.48952202,  0.121812  , ...,  1.18006101,
         0.3986462 , -0.87244756],
       [-0.40956603, -0.48952202,  0.121812  , ...,  1.18006101,
         0.43720271, -0.66992932]])

In [90]:
# 테스트, 훈련 세트 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(scaled_x, y, test_size=0.2)

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(402, 13) (101, 13)
(402,) (101,)


In [46]:
# 단순선형회귀
import sklearn.linear_model
dir(sklearn.linear_model)
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)
pred = lr.predict(X_test)
pred

array([20.70136865,  9.03045204, 30.56994686, 14.10400906, 16.73379569,
       36.4860267 , 25.30072047, 36.27997133, 13.57969272, 25.02415689,
       36.43878654, 12.58858432, 21.99944001, 21.53042363, 22.43862141,
       21.93942148, 27.06083878, 44.25895882, 17.38921311, 17.29190439,
       28.01064865, 31.70566013, 26.19507851, 22.85698794, 13.03268421,
       19.93461624, 19.95883802, 22.08168203, 26.97767987, 13.29318716,
       25.60999288, 25.15342882, 20.03544212, 32.51497766, 17.37088792,
       14.08299171, 16.61873942, 21.45144473, 20.10641537, 22.63211226,
       31.15469325, 27.06955852, 26.05541317, 25.83741536, 31.09381842,
       11.9844256 , 19.44418538, 14.35970612, 17.9028513 , 23.24591153,
       20.27332126, 20.58920552, 18.39335896, 16.19004848, 22.11641456,
       25.44158982, 37.89175296, 12.92560355, 35.35402516, 27.30788242,
       16.37858   , 19.69466168, 29.16774987, 23.33486004, 15.27888718,
       29.3438832 , 19.70228645,  5.69581522,  4.83683444, 27.35

In [49]:
# 회귀평가 -> MSE(RMSE), R2 SCORE
import sklearn.metrics
dir(sklearn.metrics)

from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, pred)
rmse = np.sqrt(mse)
r2score = r2_score(y_test, pred)

print('rmse : ',rmse)
print('r2score : ',r2score)

rmse :  4.280703582674453
r2score :  0.7567201107791426


In [65]:
# lr 절편값, 회귀계수값

print(lr.intercept_)
print(np.round(lr.coef_,2))

coef = pd.Series(np.round(lr.coef_,2), index = x.columns)
coef.sort_values()

22.67945903290544
[-1.09  1.15  0.22  0.69 -1.64  2.93 -0.17 -2.99  2.59 -1.97 -1.9   1.1
 -3.77]


LSTAT     -3.77
DIS       -2.99
TAX       -1.97
PTRATIO   -1.90
NOX       -1.64
CRIM      -1.09
AGE       -0.17
INDUS      0.22
CHAS       0.69
B          1.10
ZN         1.15
RAD        2.59
RM         2.93
dtype: float64

In [68]:
# 교차검증
from sklearn.model_selection import cross_val_score

# 분류 교차검증 지표평가의 여러 score(정확도, 재현율 등)는 값이 높을 수록 좋은모델이지만,
# 회귀 교차검증 cross_val_score은,
# MSE(오차) 값이 낮을 수록 좋은 모델이기 때문에, '음수' 를 사용하고,
# 나중에 -1을 곱하여 양수로 바꿔준다.
neg_mse_score = cross_val_score(lr, X_train, y_train, scoring="neg_mean_squared_error", cv = 5)
rmse_score = np.sqrt(-1 * (neg_mse_score))
avg_rmse = np.mean(rmse_score)

print(np.round(neg_mse_score, 2))
print(np.round(rmse_score, 2))
print(np.round(avg_rmse, 2))

[-44.72 -17.76 -17.55 -24.72 -14.41]
[6.69 4.21 4.19 4.97 3.8 ]
4.77


In [71]:
# 릿지 회귀
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score

ridge = Ridge(alpha = 10)
neg_mse_score = cross_val_score(ridge, X_train, y_train, scoring="neg_mean_squared_error", cv = 5)
rmse_scores = np.sqrt(-1 * neg_mse_score)
avg_rmse = np.mean(rmse_scores)

print(neg_mse_score)
print(rmse_scores)
print(avg_rmse)

[-45.61700454 -17.51639437 -17.87471935 -24.07310743 -14.23311173]
[6.75403617 4.18525918 4.22785044 4.90643531 3.77267965]
4.769252147981701


In [75]:
# 릿지 회귀의 alpha 파라미터 값 변화
alphas = [0, 0.1, 1, 10, 100]

for alpha in alphas : 
  ridge = Ridge(alpha = alpha)
  neg_mse_score = cross_val_score(ridge, X_train, y_train, scoring="neg_mean_squared_error", cv = 5)
  rmse_scores = np.sqrt(-1 * neg_mse_score)
  avg_rmse = np.mean(rmse_scores)

  print('alpha [{0}] 일 때 5 folds 의 평균 RMSE : {1:.3f}'.format(alpha, avg_rmse))

alpha [0] 일 때 5 folds 의 평균 RMSE : 4.772
alpha [0.1] 일 때 5 folds 의 평균 RMSE : 4.771
alpha [1] 일 때 5 folds 의 평균 RMSE : 4.770
alpha [10] 일 때 5 folds 의 평균 RMSE : 4.769
alpha [100] 일 때 5 folds 의 평균 RMSE : 4.957


In [93]:
#alpha 값의 변화에 따른 릿지 회귀 계수
coeff_df = pd.DataFrame()

for alpha in enumerate(alphas) :
  ridge = Ridge(alpha = alpha)
  ridge.fit(X_train, y_train)
  coeff = pd.Series(data=ridge.coef_, index=x.columns)
  colname = 'alpha:' + str(alpha)
  coeff_df[colname] = coeff
  coeff = coeff.sort_values(ascending=False)


ValueError: ignored

In [94]:
ridge.fit(X_train, y_train)

ValueError: ignored

In [ ]:

ridge_alphas = [0, 0.1, 1, 10, 100]
sort_column = 'alpha:' + str(ridge_alphas[0])
coeff_df.sort_values(by=sort_column, ascending=False)

In [89]:
y_train

439    12.8
487    20.6
14     18.2
290    28.5
83     22.9
       ... 
151    19.6
369    50.0
179    37.2
444    10.8
308    22.8
Name: PRICE, Length: 402, dtype: float64

In [91]:
y_train

458    14.9
270    21.1
474    13.8
403     8.3
279    35.1
       ... 
373    13.8
231    31.7
498    21.2
486    19.1
328    19.3
Name: PRICE, Length: 402, dtype: float64